In [173]:
import pandas as pd
offline = pd.read_csv('offline_train.csv', header=0)
print(offline.shape[0])

offline_notNull = offline[offline.Coupon_id.notnull()]
print(offline_notNull.shape[0])
X=offline_notNull

1754884
1053282


## c1: coupon type
1. full discount : 1
2. direct discount : 0 

In [174]:
X.loc[X.Discount_rate.str.contains(':') , 'c1'] = 1 #discount_rate contain ':' is full discount type
X.loc[~X.Discount_rate.str.contains(':'), 'c1'] = 0#discount_rate doesn't contain ':' is full discount type

C:\Users\95857\AppData\Local\Temp\ipykernel_21740\2920072700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[X.Discount_rate.str.contains(':') , 'c1'] = 1 #discount_rate contain ':' is full discount type


## c2: Coupon received historical number

In [175]:
temp = offline_notNull.groupby(['Coupon_id']).size().reset_index(name='c2')#Count the frequency of different coupon_id
X = pd.merge(X, temp, how='left', on=['Coupon_id'])

## c3: Coupon verified historical number

In [176]:
coupon_consume = offline_notNull[offline_notNull.Date.notnull()]#dataset of coupon which were consumed
temp = coupon_consume.groupby(['Coupon_id']).size().reset_index(name='c3')#Count the frequency of different coupon_id which were consumed
X = pd.merge(X, temp, how='left', on=['Coupon_id'])

##  c4: Coupon verified rate

In [178]:
X['c4']=X['c3']/X['c2']

## c5: Coupon verified interval
Date - Date_received	

In [180]:
temp= offline_notNull
X['Date'] = pd.to_datetime(X['Date'],format='%Y%m%d')
X['Date_received'] = pd.to_datetime(X['Date_received'],format='%Y%m%d')
X['c5']=X['Date'] - X['Date_received']

## c6: Coupon received date of week
0 to 6 means Monday to Sunday

In [182]:
X['c6']=X.Date_received.dt.weekday

## c7: Coupon verified date of week
0 to 6 means Monday to Sunday

In [184]:
X['c7']=X.Date.dt.weekday

## c8: Coupon frequency received by different users

In [192]:
temp = offline_notNull.groupby(['User_id', 'Coupon_id']).size().reset_index(name='c8')# frequency of different coupon received by different users
X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id'])

## c9: Coupon frequency verified by different users

In [194]:
temp = coupon_consume.groupby(['User_id', 'Coupon_id']).size().reset_index(name='c9')
X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id'])

## c10: Coupon verified rate by different users

In [196]:
X['c10']=X['c9']/X['c8']

## c11: Coupon with full discount threshold

In [199]:
# split Discount_rate into full discount threshold(c11) and discount amount(c12)
temp=pd.DataFrame((x.split(':') for x in X['Discount_rate']),index=X.index,columns=['c11','c12'])
X=pd.merge(X,temp,right_index=True, left_index=True)
# delete the direct discount data from c11
discount_types = ['0.2', '0.5', '0.6', '0.7', '0.75', '0.8', '0.85', '0.9', '0.95']
for k, v in enumerate(discount_types):
    X.loc[X['c11'] == v, 'c11'] =None

In [200]:
X

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12
0,1439408,4663,11002.0,150:20,1.0,2016-05-28,NaT,1.0,7730,15.0,0.001940,NaT,5,NaN,1,NaN,NaN,150,20
1,1439408,2632,8591.0,20:1,0.0,2016-02-17,NaT,1.0,31,4.0,0.129032,NaT,2,NaN,3,1.0,0.333333,20,1
2,1439408,2632,1078.0,20:1,0.0,2016-03-19,NaT,1.0,12,NaN,NaN,NaT,5,NaN,1,NaN,NaN,20,1
3,1439408,2632,8591.0,20:1,0.0,2016-06-13,NaT,1.0,31,4.0,0.129032,NaT,0,NaN,3,1.0,0.333333,20,1
4,1439408,2632,8591.0,20:1,0.0,2016-05-16,2016-06-13,1.0,31,4.0,0.129032,28 days,0,0.0,3,1.0,0.333333,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053277,212662,3021,3739.0,30:1,6.0,2016-05-04,2016-05-08,1.0,11,2.0,0.181818,4 days,2,6.0,2,2.0,1.000000,30,1
1053278,212662,2934,5686.0,30:5,2.0,2016-03-21,2016-03-22,1.0,10533,1611.0,0.152948,1 days,0,1.0,2,2.0,1.000000,30,5
1053279,212662,3021,3739.0,30:1,6.0,2016-05-08,2016-06-02,1.0,11,2.0,0.181818,25 days,6,3.0,2,2.0,1.000000,30,1
1053280,752472,7113,1633.0,50:10,6.0,2016-06-13,NaT,1.0,929,12.0,0.012917,NaT,0,NaN,1,NaN,NaN,50,10


## c12: Coupon with full discount rate (discount amount/ full discount threshold) 

In [201]:
#Replace the discount amount with a full discount rate
X['c12']=X['c12'].astype("Int64")/X['c11'].astype("Int64")

# Export file

In [202]:
feature=['User_id','Merchant_id','Coupon_id','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12']
coupon_feature=X[feature]
coupon_feature
coupon_feature.to_csv('coupon_feature.csv')

In [205]:
coupon_feature

,User_id,Merchant_id,Coupon_id,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12
0,1439408,4663,11002.0,1.0,7730,15.0,0.001940,NaT,5,NaN,1,NaN,NaN,150,0.133333
1,1439408,2632,8591.0,1.0,31,4.0,0.129032,NaT,2,NaN,3,1.0,0.333333,20,0.05
2,1439408,2632,1078.0,1.0,12,NaN,NaN,NaT,5,NaN,1,NaN,NaN,20,0.05
3,1439408,2632,8591.0,1.0,31,4.0,0.129032,NaT,0,NaN,3,1.0,0.333333,20,0.05
4,1439408,2632,8591.0,1.0,31,4.0,0.129032,28 days,0,0.0,3,1.0,0.333333,20,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053277,212662,3021,3739.0,1.0,11,2.0,0.181818,4 days,2,6.0,2,2.0,1.000000,30,0.033333
1053278,212662,2934,5686.0,1.0,10533,1611.0,0.152948,1 days,0,1.0,2,2.0,1.000000,30,0.166667
1053279,212662,3021,3739.0,1.0,11,2.0,0.181818,25 days,6,3.0,2,2.0,1.000000,30,0.033333
1053280,752472,7113,1633.0,1.0,929,12.0,0.012917,NaT,0,NaN,1,NaN,NaN,50,0.2
